# Tomography movie

In [1]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as color
import nugridpy.utils as utils
import sys, os, time
import inspect

# if you make changes to the ppmpy module (e.g. add your analysis methods via a pull 
# request) in the https://github.com/PPMstar/PyPPM repo you may want use that 
# updated version
#sys.path.insert(0,'/user/david/PyPPM/')
#sys.path.insert(0,'/user/PyPPM/')
# sys.path.insert(0,'/home/user/home/PyPPM/')
sys.path.insert(0, '/user/scratch14_ppmstar/bamcewan/repos/PyPPM/')
from ppmpy import ppm
print(inspect.getfile(ppm))

cb = utils.linestylecb # colours

/user/scratch14_ppmstar/bamcewan/repos/PyPPM/ppmpy/ppm.py


In [3]:
import logging
l = logging.getLogger()
l.setLevel(logging.ERROR)

In [4]:
%%bash
ls /data/ASDR/PPMstar/H-core-M25/M45/myavsbq/2650

HcoreM025Z0-BQav2650.aaa


In [5]:
dir_repo    = '/data/ASDR/PPMstar'
dir_project = 'H-core-M25/'
rprof = {}; moms = {}         # initialize dictionaries to hold rprof and moms instances

moms_dumps = {}

runs                = ['M45'] # select runs
moms_dumps[runs[0]] = 2100   # select dump numbers for moms

add_highres = False
if add_highres:
    runs.append('M35-1536')
    moms_dumps[runs[1]] = 375   

# rprof instance holds radial profiles for all dumps
# moms instance holds only one dump at a time
for run in runs:        
    path = os.path.join(dir_repo,dir_project,run)
    # radial profile:
    rprof[run] = ppm.RprofSet(os.path.join(path,'prfs'))
    moms[run] = ppm.MomsDataSet(os.path.join(path,'myavsbq'),moms_dumps[run])
print("moms and rprof dictionary created")

10000 rprof files found in '/data/ASDR/PPMstar/H-core-M25/M45/prfs/.
Dump numbers range from 0 to 9999.
10000 .aaa files found in '/data/ASDR/PPMstar/H-core-M25/M45/myavsbq/.
Dump numbers range from 0 to 9999.
moms and rprof dictionary created


In [6]:
print(moms_dumps)

{'M45': 2100}


In [7]:
runid = 'M45'

# Slice_Gui

In [8]:
thismoms = moms[runid]
thismoms.slice_gui()

Output()